## Jimmy requested eclipse timings for TIC 4707 for a range of days during 2024Q4 so if there's a B-system eclipse we might request that particular day in our proposal.

In [6]:
import astropy.table as tb
from astropy.time import Time
import astropy.units as u


In [13]:
ephemerides = tb.vstack([
    tb.Table.read(r'..\..\django\TargetDB\Data Files\Binary Parameters\Kostov 2022 Binary Parameters.csv'),
    tb.Table.read(r'..\..\django\TargetDB\Data Files\Binary Parameters\Kostov 2023 Binary Parameters.csv'),
])
ephemerides[ephemerides["Local ID"] == "TIC 470710327"]

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 470710327,A,1.104686,1765.1668,1765.721905,60,50,5.8,5.5
TIC 470710327,B,19.950922,1805.4542,--,70,--,--,--


In [14]:
tess_offset = 2457000
beg_date = Time("2024-11-09", format="iso", scale="utc").jd - tess_offset
end_date = Time("2024-11-21", format="iso", scale="utc").jd - tess_offset

for system in ephemerides[ephemerides["Local ID"] == "TIC 470710327"]:
    for member_letter, member_name in [("a", "Primary"), ("b", "Secondary")]:
        name = f"T0 {member_name}"
        if system[name] > 0:
            t0 = system[name]
            period = system["Period"]
            if not (half_duration := system[f"Duration {member_name}"] / 24 / 2):
                half_duration = 0
            mid_eclipse = t0 + int((beg_date - t0) / period) * period
            while True:
                eclipse_beg = mid_eclipse - half_duration
                eclipse_end = mid_eclipse + half_duration
                print(system["Member"], member_letter, Time(eclipse_beg + tess_offset, format="jd").iso, Time(eclipse_end + tess_offset, format="jd").iso, t0, period)
                mid_eclipse += period
                if mid_eclipse > end_date:
                    break


A a 2024-11-08 15:04:03.533 2024-11-08 20:52:03.533 1765.1668 1.104686
A a 2024-11-09 17:34:48.403 2024-11-09 23:22:48.403 1765.1668 1.104686
A a 2024-11-10 20:05:33.274 2024-11-11 01:53:33.274 1765.1668 1.104686
A a 2024-11-11 22:36:18.144 2024-11-12 04:24:18.144 1765.1668 1.104686
A a 2024-11-13 01:07:03.014 2024-11-13 06:55:03.014 1765.1668 1.104686
A a 2024-11-14 03:37:47.885 2024-11-14 09:25:47.885 1765.1668 1.104686
A a 2024-11-15 06:08:32.755 2024-11-15 11:56:32.755 1765.1668 1.104686
A a 2024-11-16 08:39:17.626 2024-11-16 14:27:17.626 1765.1668 1.104686
A a 2024-11-17 11:10:02.496 2024-11-17 16:58:02.496 1765.1668 1.104686
A a 2024-11-18 13:40:47.366 2024-11-18 19:28:47.366 1765.1668 1.104686
A a 2024-11-19 16:11:32.237 2024-11-19 21:59:32.237 1765.1668 1.104686
A a 2024-11-20 18:42:17.107 2024-11-21 00:30:17.107 1765.1668 1.104686
A b 2024-11-08 02:01:39.734 2024-11-08 07:31:39.734 1765.721905 1.104686
A b 2024-11-09 04:32:24.605 2024-11-09 10:02:24.605 1765.721905 1.104686
A 

In [15]:
from astropy.table.row import Row
from astropy.time import Time
import astropy.units as u

def calc_eclipse_timings(
    t0: Time = None,
    period: u.Quantity = None,
    duration: u.Quantity = None,
    beg_time: Time = None,
    end_time: Time = None,
    count: int = 0,
    row = None,
) -> list[tuple[Time, Time]]:
    """Given an initial time (t0), a period and optionally a duration, return a list of eclipses.
    The results can be a fixed number of eclipses, or all eclipses falling between a beginning and end time.
    Period & duration are Astropy Quantity, t0, beg_time & end_time are Astropy Time objects, and count is an int.
    """
    # process & validate inputs
    if isinstance(row, Row):
        t0 = row["T0"]
        period = row["Period"]
        if "Duration" in row.columns:
            duration = row["Duration"]
    if beg_time is None:
        raise ValueError("Must specify begin time")
    if count > 0:
        if end_time is not None:
            raise ValueError("Cannot specify both a count and an end time")
        use_count = True
    else:
        if count < 0:
            raise ValueError("Count cannot be negative")
        if end_time is None:
            raise ValueError("If count is zero, end time must be specified")
        use_count = False


    if duration is not None:
        half_duration = duration / 2
    t = t0 + (round((beg_time.jd - t0.jd) / period.value) * period)
    while t - half_duration < beg_time:
        t += period
    eclipses = []
    while True:
        if duration is not None:
            eclipse_beg = t - half_duration
            eclipse_end = t + half_duration
        else:
            eclipse_beg = t
            eclipse_end = t
        eclipses.append((eclipse_beg, eclipse_end))
        print(eclipse_beg.iso, "   ", eclipse_end.iso)
        t += period
        if use_count:
            if len(eclipses) >= count:
                break
        else:
            if end_time < t + half_duration:
                break
    return eclipses


beg_time = Time("2024-11-10", format="iso", scale="utc")
end_time = Time("2024-11-21", format="iso", scale="utc")

foo = QTable(names=["T0", "Period", "Duration"], dtype=[Time, u.Quantity, u.Quantity])
# foo.add_row([1,2 * u.day,3])
foo.add_row([Time(1765.1668 + tess_offset, format="jd"), 1.104686 * u.day, 5.8 * u.hour])
calc_eclipse_timings(row=foo[0], beg_time=beg_time, count=1)

# print("Aa")
# calc_eclipse_timings(Time(1765.1668 + tess_offset, format="jd"), 1.104686 * u.day, 5.8 * u.hour, beg_time, end_time)
# print("Ab")
# calc_eclipse_timings(Time(1765.721905 + tess_offset, format="jd"), 1.104686 * u.day, 5.5 * u.hour, beg_time, end_time)
# print("Ba")
# calc_eclipse_timings(Time(1805.4542 + tess_offset, format="jd"), 19.950922 * u.day, None, beg_time, end_time)

2024-11-10 20:05:33.274     2024-11-11 01:53:33.274


[(<Time object: scale='utc' format='jd' value=2460625.3371906662>,
  <Time object: scale='utc' format='jd' value=2460625.578857333>)]

## After catching a bug in the one-off code (while testing the above function), do some manual cross-checking to make sure I'm not jerking Jimmy around with bad answers.

## Convert CSV file of Kostov ephemerides to something easier to manage, and in standard units.

In [38]:
from astropy.table import QTable

better_ephem = QTable(names=["Local ID", "System", "Member", "T0", "Period", "Duration", "Depth"],
                      units={"Period": u.day, "Duration": u.hour},
                      dtype=[str, str, str, float, u.Quantity, u.Quantity, float],
                      )
# better_ephem["Period"].unit = u.day
# better_ephem["Duration"].unit = u.hour

for ephem in ephemerides:
    for letter, word in [("a", "Primary"), ("b", "Secondary")]:
        if not ephem[f"T0 {word}"] or ephem["Period"] < 0:
            continue
        if ephem[f"Duration {word}"]:
            duration = ephem[f"Duration {word}"]# * u.hour
        else:
            duration = 0
        better_ephem.add_row([
            ephem["Local ID"],
            ephem["Member"],
            letter,
            Time(ephem[f"T0 {word}"] + tess_offset, format="jd").jd,
            ephem["Period"],# * u.day,
            duration,
            ephem[f"Depth {word}"] / 1000,
            ],
            # mask = [False, False, False, False, False, not (ephem[f"Duration {word}"] > 0), False],
            )

better_ephem.write("Kostov ephemerides.ecsv", overwrite=True)
better_ephem

Local ID,System,Member,T0,Period,Duration,Depth
,,,,d,h,
str14,str1,str1,float64,object,object,float64
TIC 9493888,A,a,2458816.2345,2.098992,2.9,0.146
TIC 9493888,A,b,2458817.283786,2.098992,2.7,0.117
TIC 25818450,A,a,2458769.9109,10.132402,0,0.012
TIC 25818450,A,b,2458776.391584,10.132402,0,0.009
TIC 27543409,A,a,2458493.1001,2.122862,0,0.05
TIC 27543409,A,b,2458494.153889,2.122862,0,0.015
TIC 31928452,A,a,2458337.9129,2.8823,2.1,0.03
TIC 31928452,A,b,2458339.358085,2.8823,1.6,0.025


In [37]:
better_ephem[0]["Duration"]


2.9

In [43]:
tics_of_interest = [
    ("", "TIC 344541836"),
    ("", "TIC 278465736"),
    ("", "TIC 161043618"),
    ("", "TIC 367448265"),
    ("", "TIC 470710327"),
]

beg_time = Time("2024-09-11", format="iso", scale="utc")
end_time = Time("2024-10-23", format="iso", scale="utc")
# beg_time = Time("2024-08-23", format="iso", scale="utc")
# end_time = Time("2024-09-11", format="iso", scale="utc")

for _, tic_of_interest in tics_of_interest:
    print()
    print(tic_of_interest)
    for ephem in better_ephem[better_ephem["Local ID"] == tic_of_interest]:
        if ephem["Duration"] is None:
            print(f"  {ephem["System"]}{ephem["Member"]}, Period={ephem["Period"]:9.5f}, Duration=unknown, Depth={ephem["Depth"]}")
        else:
            print(f"  {ephem["System"]}{ephem["Member"]}, Period={ephem["Period"]:9.5f}, Duration={ephem["Duration"]:7.4f}, Depth={ephem["Depth"]}")
        calc_eclipse_timings(Time(ephem["T0"], format="jd"), ephem["Period"] * u.day, ephem["Duration"] * u.hour, beg_time, end_time)


TIC 344541836
  Aa, Period=  2.40993, Duration= 3.4000, Depth=0.045
2024-09-11 02:23:46.406     2024-09-11 05:47:46.406
2024-09-13 12:14:04.531     2024-09-13 15:38:04.531
2024-09-15 22:04:22.656     2024-09-16 01:28:22.656
2024-09-18 07:54:40.781     2024-09-18 11:18:40.781
2024-09-20 17:44:58.906     2024-09-20 21:08:58.906
2024-09-23 03:35:17.030     2024-09-23 06:59:17.030
2024-09-25 13:25:35.155     2024-09-25 16:49:35.155
2024-09-27 23:15:53.280     2024-09-28 02:39:53.280
2024-09-30 09:06:11.405     2024-09-30 12:30:11.405
2024-10-02 18:56:29.530     2024-10-02 22:20:29.530
2024-10-05 04:46:47.654     2024-10-05 08:10:47.654
2024-10-07 14:37:05.779     2024-10-07 18:01:05.779
2024-10-10 00:27:23.904     2024-10-10 03:51:23.904
2024-10-12 10:17:42.029     2024-10-12 13:41:42.029
2024-10-14 20:08:00.154     2024-10-14 23:32:00.154
2024-10-17 05:58:18.278     2024-10-17 09:22:18.278
2024-10-19 15:48:36.403     2024-10-19 19:12:36.403
2024-10-22 01:38:54.528     2024-10-22 05:02:54